In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn import linear_model, preprocessing, model_selection

In [ ]:
iron = pd.read_csv('ironslag.csv')
iron.head()

In [ ]:
plt.scatter(iron.chemical, iron.magnetic)

In [ ]:
# sklearn requires our predictor variables to be in a two dimensional array
# reshape to have 1 column
# the -1 in reshape means I don't want to figure out all the necessary dimensions
# i want 1 column, and numpy, you figure out how many rows I need
X = iron.chemical.values.reshape(-1,1)
X.shape

In [ ]:
y = iron.magnetic
y.shape

In [ ]:
np.corrcoef(iron.chemical.values, iron.magnetic.values)[0,1] ** 2

# Fit a linear model between x and y

In [ ]:
linear = linear_model.LinearRegression()

In [ ]:
linear.fit(X, y)

In [ ]:
# linear.score is the R^2 value
# how much error is reduced from no model (variance or MSE)
# vs having the regression model
linear.score(X,y)

In [ ]:
x_predict = np.arange(10,33).reshape(-1,1)

In [ ]:
lin_y_hat = linear.predict(x_predict)

In [ ]:
plt.scatter(X,y)
plt.plot(x_predict, lin_y_hat, c = 'red')

### Cross Validation

In [ ]:
# shuffle split says 'shuffle the data' and split it into 5 equal parts
cv = model_selection.ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cv_linear = model_selection.cross_val_score(linear, X, y, cv = cv)
print(cv_linear)
print(np.mean(cv_linear))

Technically, the above is all you need to do. But I went ahead and wrote this loop which fits the model on the training data, and makes predictions for the validation data. 

In each plot, the light blue dots are the training data.

The green dots are the validation data.

The flat green line is the mean of the validation data. That would be the prediction if no model was fit.

The red line is the linear model that was trained on the training data. We hope that the red line does a better job of predicting the green points than the green line. In some cases, it does not, and we actually get a negative cross-validation score.

In [ ]:
for train_index, test_index in cv.split(X):
    # create a subset of the data using the training cases in cross validation
    tX = X[train_index, : ]
    ty = y[train_index]
    
    # initialize and fite a new linear regression model
    clin = linear_model.LinearRegression()
    
    # fit only on the training cases
    clin.fit(tX, ty)
    
    # create a subset for the test cases
    testX = X[test_index, :]
    testy = y[test_index]
    
    # plot the training data in blue and test cases in green
    plt.scatter(tX,ty, c = 'blue', alpha = 0.1)
    plt.scatter(testX, testy, c = 'green')
    
    # plot the fitted prediction line based on the training data
    plt.plot(x_predict, clin.predict(x_predict), c = 'red')
    
    # plot the mean of the test cases to show what having no model looks like
    plt.plot(x_predict, np.repeat(np.mean(testy), len(x_predict)), c = 'lightgreen')
    
    plt.show()
    
    # the MS of having no model = variance of the test data
    mse = np.var(testy)
    print(mse)
    
    # the MS regression
    msr = sum((testy - clin.predict(testX))**2)/len(testy)
    print( msr )
    
    # the score is the proportion of reduction by having regression
    red = (mse - msr)/mse
    print(red)

# Polynomial fit - quadratic

In [ ]:
# preprocessing polynomial features creates a polynomial based on X
poly2 = preprocessing.PolynomialFeatures(2)

In [ ]:
polyX = poly2.fit_transform(X)

In [ ]:
poly2reg = linear_model.LinearRegression(fit_intercept = False)

In [ ]:
poly2reg.fit(polyX, y)

In [ ]:
poly2reg.score(polyX,y)

In [ ]:
poly2_X_new = poly2.fit_transform(x_predict)
poly2_y_hat = poly2reg.predict(poly2_X_new)

In [ ]:
plt.scatter(X,y)
plt.plot(x_predict, poly2_y_hat, c = 'red')

In [ ]:
cv_quad = model_selection.cross_val_score(poly2reg, polyX, y, cv=cv)
print(cv_quad)
print(np.mean(cv_quad))

In [ ]:
def polycv(degree, X, y, train_index, test_index):
    
    # create a subset of the data using the training cases in cross validation
    tX = X[train_index, : ]
    ty = y[train_index]
    
    # create a subset for the test cases
    testX = X[test_index, :]
    testy = y[test_index]
    
    poly = preprocessing.PolynomialFeatures(degree)
    polytX = poly.fit_transform(tX)
    
    # initialize and fite a new linear regression model
    clin = linear_model.LinearRegression()
    
    # fit only on the training cases
    clin.fit(polytX, ty)
    
    # plot the training data in blue and test cases in green
    plt.scatter(tX,ty, c = 'blue', alpha = 0.1)
    plt.scatter(testX, testy, c = 'green')
    
    # plot the fitted prediction line based on the training data
    plt.plot(x_predict, clin.predict(poly.fit_transform(x_predict)), c = 'red')
    
    # plot the mean of the test cases to show what having no model looks like
    plt.plot(x_predict, np.repeat(np.mean(testy), len(x_predict)), c = 'lightgreen')
    
    plt.show()
    
    # the MS of having no model = variance of the test data
    mse = np.var(testy)
    print(mse)
    
    # the MS regression
    msr = sum((testy - clin.predict(poly.fit_transform(testX)))**2)/len(testy)
    print( msr )
    
    # the score is the proportion of reduction by having regression
    red = (mse - msr)/mse
    print(red)

for train_index, test_index in cv.split(X):
    polycv(2, X, y, train_index, test_index)

# cubic fit

In [ ]:
poly3 = preprocessing.PolynomialFeatures(3)

In [ ]:
poly3X = poly3.fit_transform(X)
poly3reg = linear_model.LinearRegression(fit_intercept = False)
poly3reg.fit(poly3X, y)
print(poly3reg.score(poly3X,y))

The R^2 value of the cubic fit is better, but we will see with cross validation that it is not a better model. It is overfitting our data.

In [ ]:
poly3_X_new = poly3.fit_transform(x_predict)
poly3_y_hat = poly3reg.predict(poly3_X_new)

In [ ]:
plt.scatter(X,y)
plt.plot(x_predict, poly3_y_hat, c = 'red')

In [ ]:
cv_cube = model_selection.cross_val_score(poly3reg, poly3X, y, cv=cv)
print(cv_cube)
print(np.mean(cv_cube))

In [ ]:
for train_index, test_index in cv.split(X):
    polycv(3, X, y, train_index, test_index)

# fitting higher order polynomials:

In [ ]:
# degree4
poly4 = preprocessing.PolynomialFeatures(4)
poly4X = poly4.fit_transform(X)
poly4reg = linear_model.LinearRegression(fit_intercept = False)

cv_4th = model_selection.cross_val_score(poly4reg, poly4X, y, cv=cv)
print(cv_4th)
print(np.mean(cv_4th))

In [ ]:
for train_index, test_index in cv.split(X):
    polycv(4, X, y, train_index, test_index)

In [ ]:
# degree 5
poly5 = preprocessing.PolynomialFeatures(5)
poly5X = poly5.fit_transform(X)
poly5reg = linear_model.LinearRegression(fit_intercept = False)

cv_5th = model_selection.cross_val_score(poly5reg, poly5X, y, cv=cv)
print(cv_5th)
print(np.mean(cv_5th))

In [ ]:
for train_index, test_index in cv.split(X):
    polycv(5, X, y, train_index, test_index)